Paper on different types of normalization:

https://genomebiology.biomedcentral.com/articles/10.1186/s13059-021-02568-9#Sec17

Article of different types of feature importance in RandomForestRegressor

https://mljar.com/blog/feature-importance-in-random-forest/

SHAP values show how each feature affects each final prediction, the significance of each feature compared to others, and the model's reliance on the interaction between features.


In [5]:
!pip install shap

  Obtaining dependency information for shap from https://files.pythonhosted.org/packages/f5/fc/e81722d6bec4fcba46e46ef895eddaeab0027ac71e78fc35ef351fac5fe4/shap-0.43.0-cp311-cp311-win_amd64.whl.metadata
  Using cached slicer-0.0.7-py3-none-any.whl (14 kB)
  Obtaining dependency information for numba from https://files.pythonhosted.org/packages/cd/59/5dd8a3059997ec1daf6f9f7c9b1aef7f0a9e23e1334a5774eae65cae6fd0/numba-0.58.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for cloudpickle from https://files.pythonhosted.org/packages/96/43/dae06432d0c4b1dc9e9149ad37b4ca8384cf6eb7700cd9215b177b914f0a/cloudpickle-3.0.0-py3-none-any.whl.metadata
  Obtaining dependency information for llvmlite<0.42,>=0.41.0dev0 from https://files.pythonhosted.org/packages/14/fe/d3a9c921a2adad2e9f24693754983f290e0dac9410666e802b9dba4d0218/llvmlite-0.41.1-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/447.3 kB ? eta -:--:--
   ------------------------ ----


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
# Load your data
import pandas as pd

expression_file = r'C:\Users\Neel Patel\Documents\Github Repositories\Machine-Learning-Biosciences-Final-Project\Project1\100_mr_50_cond\simulated_noNoise.txt'
ground_truth_file = r'C:\Users\Neel Patel\Documents\Github Repositories\Machine-Learning-Biosciences-Final-Project\Project1\100_mr_50_cond\bipartite_GRN.csv'

#expression_file = r'C:\Users\neelh\Documents\Github Repositories\Machine-Learning-Biosciences-Final-Project\Project1\100_mr_50_cond\simulated_noNoise.txt'
#ground_truth_file = r'C:\Users\neelh\Documents\Github Repositories\Machine-Learning-Biosciences-Final-Project\Project1\100_mr_50_cond\bipartite_GRN.csv'


data = pd.read_csv(expression_file, sep="\t")
tg = data.iloc[:,0:100]

tg.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,3.26,7.52,3.05,5.29,8.00,4.02,3.99,3.89,6.73,3.71,...,7.33,8.11,7.60,4.43,6.23,4.70,4.85,5.48,3.33,3.530
1,7.72,9.75,3.25,3.18,4.19,6.19,4.68,2.57,6.26,4.79,...,3.93,6.77,2.27,7.52,7.17,5.92,7.69,6.64,5.46,6.160
2,1.31,4.23,2.43,3.72,5.78,2.55,6.59,1.76,9.21,4.24,...,6.04,8.59,7.01,6.51,3.15,1.93,4.82,6.87,4.63,0.816
3,6.87,5.70,2.42,3.76,4.91,6.83,3.98,2.37,3.81,3.33,...,3.97,3.12,6.96,6.22,5.06,4.95,5.84,3.25,2.56,6.930
4,2.33,5.48,2.56,3.06,2.11,5.91,5.34,3.62,5.14,1.55,...,4.11,5.38,6.47,8.18,5.61,5.22,9.52,5.88,6.19,3.290


In [2]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import roc_auc_score

def load_data(expression_file):
    data = pd.read_csv(expression_file, sep="\t")
    tf_expression = data.iloc[:, :100].values  
    tg_expression = data.iloc[:, 100:].values  
    return tf_expression, tg_expression

def train_models(tf_expression, tg_expression):
    models = []
    for i in range(tg_expression.shape[1]):
        model = RandomForestRegressor(n_estimators=100,max_features=None)
        model.fit(tf_expression, tg_expression[:, i])
        models.append(model)
    return models

def infer_grn(models, threshold=0.01):
    grn_edges = []
    for i, model in enumerate(models):
        importance = model.feature_importances_
        regulators = np.where(importance > threshold)[0]
        for reg in regulators:
            grn_edges.append((reg, i+100))  # TF ids are 0-99, TG ids are 100-199
      
    return grn_edges

def evaluate_grn(predicted_edges, ground_truth_file):
    ground_truth = pd.read_csv(ground_truth_file, header=None)
    ground_truth.head()
    ground_truth_set = set(tuple(x) for x in ground_truth.values)
    predicted_set = set(predicted_edges) #convert into a set to get the unique ordered pairs (also can't edit the sets themselves)
    
   
    #Accuracy (tn calcuation) doesn't seem to be meaningful because there will be more non-regulatory edges then actual edges
    
    tp = len(predicted_set & ground_truth_set)
    fp = len(predicted_set - ground_truth_set)
    fn = len(ground_truth_set - predicted_set)
    
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    print("HERE ARE TP,FP,FN")  
    print(tp,fp,fn)
    return precision, recall


# Load your data and adjust file path as necessary!
tf_expression, tg_expression = load_data(expression_file)

# Train a model for each target gene
models = train_models(tf_expression, tg_expression)

# Infer the gene regulatory network
predicted_grn_edges = infer_grn(models)

# Evaluate the GRN
precision, recall= evaluate_grn(predicted_grn_edges, ground_truth_file)

print(f"Precision: {precision}, Recall: {recall}")


here are the grn edges
[(18, 100), (29, 100), (39, 100), (44, 100), (45, 100), (52, 100), (54, 100), (56, 100), (62, 100), (69, 100), (73, 100), (74, 100), (77, 100), (80, 100), (81, 100), (89, 100), (90, 100), (92, 100), (93, 100), (98, 100), (22, 101), (44, 101), (50, 101), (61, 101), (80, 101), (90, 101), (94, 101), (99, 101), (6, 102), (10, 102), (13, 102), (26, 102), (30, 102), (32, 102), (36, 102), (37, 102), (38, 102), (56, 102), (59, 102), (73, 102), (74, 102), (77, 102), (14, 103), (23, 103), (32, 103), (35, 103), (45, 103), (52, 103), (54, 103), (58, 103), (64, 103), (75, 103), (76, 103), (80, 103), (83, 103), (86, 103), (91, 103), (95, 103), (10, 104), (23, 104), (24, 104), (25, 104), (29, 104), (39, 104), (45, 104), (57, 104), (58, 104), (59, 104), (72, 104), (83, 104), (84, 104), (85, 104), (1, 105), (3, 105), (4, 105), (9, 105), (11, 105), (16, 105), (18, 105), (19, 105), (20, 105), (26, 105), (27, 105), (36, 105), (38, 105), (39, 105), (43, 105), (53, 105), (63, 105), (6

# Approach 1 - Correlation Pruning

In [4]:
import numpy as np
import pandas as pd
from scipy.stats import pearsonr
from sklearn.ensemble import RandomForestRegressor

def correlation_pruning(preliminary_network, expression_data, threshold):
    pruned_network = []
    for tf, tg in preliminary_network:
        correlation, _ = pearsonr(expression_data[:, tf], expression_data[:, tg])  # tg is already referenced from indices 100 onward
        if abs(correlation) > threshold:
            pruned_network.append((tf, tg))
    return pruned_network


expression_file = r'C:\Users\Neel Patel\Documents\Github Repositories\Machine-Learning-Biosciences-Final-Project\Project1\100_mr_50_cond\simulated_noNoise.txt'
ground_truth_file = r'C:\Users\Neel Patel\Documents\Github Repositories\Machine-Learning-Biosciences-Final-Project\Project1\100_mr_50_cond\bipartite_GRN.csv'

# Assume the rest of the functions like train_models and load_data are defined as above
expression_data = pd.read_csv(expression_file, sep="\t", header=None).values

# Load your data
tf_expression, tg_expression = load_data(expression_file)

# Train a model for each target gene and construct preliminary network
models = train_models(tf_expression, tg_expression)
preliminary_network = infer_grn(models)  # Modify this function to output (tf, tg) pairs

# Prune the network based on correlation
final_network = correlation_pruning(preliminary_network, expression_data,threshold=0.2)

# Evaluate the final GRN
precision, recall = evaluate_grn(final_network, r'C:\Users\Neel Patel\Documents\Github Repositories\Machine-Learning-Biosciences-Final-Project\Project1\100_mr_50_cond\bipartite_GRN.csv')

print(f"Precision: {precision}, Recall: {recall}")

#THIS DOES NOT WORK REALLY WELL!

here are the grn edges
[(10, 100), (25, 100), (29, 100), (44, 100), (45, 100), (52, 100), (54, 100), (56, 100), (57, 100), (62, 100), (69, 100), (74, 100), (77, 100), (80, 100), (81, 100), (89, 100), (90, 100), (93, 100), (98, 100), (13, 101), (22, 101), (50, 101), (90, 101), (94, 101), (99, 101), (6, 102), (10, 102), (24, 102), (26, 102), (30, 102), (36, 102), (38, 102), (56, 102), (58, 102), (59, 102), (74, 102), (77, 102), (82, 102), (83, 102), (6, 103), (14, 103), (23, 103), (31, 103), (32, 103), (43, 103), (44, 103), (45, 103), (52, 103), (54, 103), (58, 103), (64, 103), (75, 103), (83, 103), (91, 103), (9, 104), (10, 104), (24, 104), (29, 104), (39, 104), (45, 104), (46, 104), (58, 104), (72, 104), (81, 104), (83, 104), (85, 104), (1, 105), (4, 105), (11, 105), (18, 105), (19, 105), (20, 105), (27, 105), (36, 105), (38, 105), (43, 105), (51, 105), (53, 105), (63, 105), (66, 105), (71, 105), (73, 105), (79, 105), (85, 105), (87, 105), (88, 105), (91, 105), (96, 105), (97, 105), (9

## Approach 2 - Shapley Feature Importance


In [12]:
import numpy as np
import pandas as pd
import shap
from sklearn.ensemble import RandomForestRegressor

def train_models_with_shap(tf_expression, tg_expression):
    models = []
    shap_values_list = []
    for i in range(tg_expression.shape[1]):
        model = RandomForestRegressor(n_estimators=100)
        model.fit(tf_expression, tg_expression[:, i])

        # Calculate SHAP values
        explainer = shap.TreeExplainer(model)
        shap_values = explainer.shap_values(tf_expression)
        shap_values_list.append(shap_values)

        models.append(model)
    return models, shap_values_list

def infer_grn_with_shap(models, shap_values_list, threshold):
    grn_edges = []
    for i, (model, shap_values) in enumerate(zip(models, shap_values_list)):
        # Use mean absolute SHAP values as feature importance
        importance = np.abs(shap_values).mean(axis=0) #compute mean shaps values
        regulators = np.where(importance > threshold)[0]
        for reg in regulators:
            grn_edges.append((reg, i+100))  # TF ids are 0-99, TG ids are 100-199
    return grn_edges

# Load your data
#expression_file = r'C:\Users\neelh\Documents\Github Repositories\Machine-Learning-Biosciences-Final-Project\Project1\100_mr_50_cond\simulated_noNoise.txt'
#ground_truth_file = r'C:\Users\neelh\Documents\Github Repositories\Machine-Learning-Biosciences-Final-Project\Project1\100_mr_50_cond\bipartite_GRN.csv'
expression_file = r'C:\Users\Neel Patel\Documents\Github Repositories\Machine-Learning-Biosciences-Final-Project\Project1\100_mr_50_cond\simulated_noNoise.txt'
ground_truth_file = r'C:\Users\Neel Patel\Documents\Github Repositories\Machine-Learning-Biosciences-Final-Project\Project1\100_mr_50_cond\bipartite_GRN.csv'

tf_expression, tg_expression = load_data(expression_file)

# Train models and calculate SHAP values
models, shap_values_list = train_models_with_shap(tf_expression, tg_expression)

# Infer the gene regulatory network using SHAP values
predicted_grn_edges = infer_grn_with_shap(models, shap_values_list,threshold=0.02)

# Evaluate the GRN
precision, recall = evaluate_grn(predicted_grn_edges, ground_truth_file)

print(f"Precision: {precision}, Recall: {recall}")


here are the metrics for groundtruthset
{(33, 164), (72, 164), (10, 198), (45, 165), (46, 139), (58, 104), (89, 171), (98, 149), (7, 138), (42, 114), (26, 160), (82, 168), (72, 132), (87, 130), (96, 108), (64, 137), (76, 102), (34, 142), (66, 192), (68, 134), (98, 135), (78, 166), (3, 172), (7, 133), (71, 136), (90, 158), (60, 126), (92, 195), (19, 134), (71, 172), (83, 137), (41, 131), (95, 111), (53, 105), (87, 116), (85, 173), (88, 117), (14, 140), (66, 178), (14, 158), (43, 194), (90, 126), (86, 183), (59, 166), (79, 144), (98, 157), (30, 102), (42, 104), (28, 169), (63, 136), (30, 111), (55, 195), (19, 120), (82, 158), (58, 197), (72, 104), (84, 106), (40, 179), (21, 175), (6, 186), (81, 198), (73, 194), (34, 123), (23, 193), (56, 141), (74, 177), (78, 147), (86, 160), (16, 172), (1, 183), (98, 143), (67, 174), (48, 133), (55, 181), (28, 164), (83, 100), (59, 179), (8, 106), (72, 128), (61, 197), (95, 110), (45, 100), (51, 152), (51, 161), (4, 184), (35, 162), (89, 142), (98, 129)

## Iteration 2 of Shapley Feature Importance - correlation pruning to get rid of the "junk" edges

In [10]:
# Load your data
expression_file = r'C:\Users\Neel Patel\Documents\Github Repositories\Machine-Learning-Biosciences-Final-Project\Project1\100_mr_50_cond\simulated_noNoise.txt'
ground_truth_file = r'C:\Users\Neel Patel\Documents\Github Repositories\Machine-Learning-Biosciences-Final-Project\Project1\100_mr_50_cond\bipartite_GRN.csv'
expression_data = pd.read_csv(expression_file, sep="\t", header=None).values

tf_expression, tg_expression = load_data(expression_file)

# Train models and calculate SHAP values
models, shap_values_list = train_models_with_shap(tf_expression, tg_expression)

# Infer the gene regulatory network using SHAP values
predicted_grn_edges = infer_grn_with_shap(models, shap_values_list,threshold = 0.05)

shap_network_pruned = correlation_pruning(predicted_grn_edges,expression_data,threshold=0.1)

precision, recall = evaluate_grn(shap_network_pruned, ground_truth_file)

print(f"Precision: {precision}, Recall: {recall}")

here are the metrics for groundtruthset
{(33, 164), (72, 164), (10, 198), (45, 165), (46, 139), (58, 104), (89, 171), (98, 149), (7, 138), (42, 114), (26, 160), (82, 168), (72, 132), (87, 130), (96, 108), (64, 137), (76, 102), (34, 142), (66, 192), (68, 134), (98, 135), (78, 166), (3, 172), (7, 133), (71, 136), (90, 158), (60, 126), (92, 195), (19, 134), (71, 172), (83, 137), (41, 131), (95, 111), (53, 105), (87, 116), (85, 173), (88, 117), (14, 140), (66, 178), (14, 158), (43, 194), (90, 126), (86, 183), (59, 166), (79, 144), (98, 157), (30, 102), (42, 104), (28, 169), (63, 136), (30, 111), (55, 195), (19, 120), (82, 158), (58, 197), (72, 104), (84, 106), (40, 179), (21, 175), (6, 186), (81, 198), (73, 194), (34, 123), (23, 193), (56, 141), (74, 177), (78, 147), (86, 160), (16, 172), (1, 183), (98, 143), (67, 174), (48, 133), (55, 181), (28, 164), (83, 100), (59, 179), (8, 106), (72, 128), (61, 197), (95, 110), (45, 100), (51, 152), (51, 161), (4, 184), (35, 162), (89, 142), (98, 129)

In [ ]:
#Create array of all possible GRN edges.

#TF * Genes = 100*100 = 10000 (for this particular file)

#create true labels array where 1 is in the groundtruth and 0 if not 



